In [1]:
import    datetime
import    email
import    imaplib
import    mailbox
import    re
import    pandas      as   pd

In [2]:
import     sys, os
import     random
import     string
import     mysql.connector

In [3]:
folder_name = r'D:\Consulting\Matrimony\Codes'
sys.path.insert(0, folder_name) 

In [4]:
from       db_Class_210821_matrimony  import  Member

In [5]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer    = PorterStemmer() 

In [6]:
import  smtplib
from    email.mime.multipart import     MIMEMultipart
from    email.mime.text      import     MIMEText

In [7]:
### Function to clean text port
def preprocess(sentence):
    sentence         =   str(sentence)
    sentence         =   sentence.lower()
    sentence         =   sentence.replace('{html}',"") 
    cleanr           =   re.compile('<.*?>')
    cleantext        =   re.sub(cleanr, '', sentence)
    rem_url          =   re.sub(r'http\S+', '',cleantext)
    #rem_num         =   re.sub('[0-9]+', '', rem_url)
    #tokenizer        =   RegexpTokenizer(r'\w+')
    pattern          =   r'\S+@[^\s.]+\.[a-zA-Z]+|\w+|[^\w\s]'
    tokenizer        =   RegexpTokenizer(pattern)
    tokens           =   tokenizer.tokenize(rem_url)  
    filtered_words   =   [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words       =   [stemmer.stem(w) for w in filtered_words]
    lemma_words      =   [lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

In [8]:
date = (datetime.date.today() - datetime.timedelta(1)).strftime("%d-%b-%Y")

In [9]:
regex         =   re.compile(r'b\"[\n\r\t\*]')
regex1        =   re.compile(r'delivering your message to')

In [10]:
def check_pattern(string):
    if re.search(r'(?<!\w){}(?!\w)'.format(re.escape('re:')), string.lower()):
       return 'Reply'
    elif   re.search(r'(?<!\w){}(?!\w)'.format(re.escape('fwd:')), string.lower()):
       return 'Forward'  
    elif   'delivery status notification (failure)' in string.lower():
       return 'Bounced' 
    else:
       return 'General'  
    

In [11]:
EMAIL_ACCOUNT     =    "pvs1301@gmail.com"
sender_address    =    "pvs1301@gmail.com"
PASSWORD          =    "KrupaGS@1302"
sender_pass       =    "KrupaGS@1302"

In [12]:
###
mail              =     imaplib.IMAP4_SSL('imap.gmail.com')
mail.login(EMAIL_ACCOUNT, PASSWORD)
mail.list()
mail.select('inbox')
# result, data     =     mail.uid('search', '(SINCE "21-Aug-2021")', "ALL") # (ALL/UNSEEN)
result, data      =     mail.uid('search', '(SENTSINCE {date})'.format(date=date), "ALL") # (ALL/UNSEEN)

i                 =     len(data[0].split())
mails_df          =     pd.DataFrame()
txt_df            =     pd.DataFrame()
for x in range(i):
    
    latest_email_uid     =  data[0].split()[x]
    result, email_data   =  mail.uid('fetch', latest_email_uid, '(RFC822)')
    # result, email_data = conn.store(num,'-FLAGS','\\Seen') 
    # this might work to set flag to seen, if it doesn't already
    raw_email            =  email_data[0][1]
    raw_email_df         =  pd.DataFrame( {'Text' : raw_email}, index = [0])
    raw_email_df['cleanText'] = raw_email_df['Text'].map(lambda s:preprocess(s))

    txt_df               =   txt_df.append(raw_email_df, ignore_index = False)  
    
    s2                   =   regex1.sub("", raw_email_df['cleanText'].to_string()); 

    try:
        raw_email_string       =  raw_email.decode('latin1')
        email_message          =  email.message_from_string(raw_email_string)
        ### Header Details
        date_tuple = email.utils.parsedate_tz(email_message['Date'])
        if date_tuple:
            local_date         = datetime.datetime.fromtimestamp(email.utils.mktime_tz(date_tuple))
            local_message_date = "%s" %(str(local_date.strftime("%a, %d %b %Y %H:%M:%S")))

        email_from  = str(email.header.make_header(email.header.decode_header(email_message['From'])))
        email_to    = str(email.header.make_header(email.header.decode_header(email_message['To'])))
        subject     = str(email.header.make_header(email.header.decode_header(email_message['Subject'])))

        category    = check_pattern(subject)

        ### Body details

        for part in email_message.walk():
                if part.get_content_type() == "text/plain":
                    body         =   part.get_payload(decode=True)
                    try:
                        Profile_ID   =   re.search(r'Profile ID:(.*) in Matrimony.com', subject).group(1).strip()
                    except:
                        Profile_ID   =   ' '
                    mail_df1     =   pd.DataFrame({'From': email_from,\
                                                   'To': email_to,\
                                                   'Date': local_message_date,\
                                                   'Subject': subject,\
                                                   'Profile ID': Profile_ID,\
                                                   'Category' : category}, index = [0])
                    mails_df     =   mails_df.append(mail_df1, ignore_index = False)            
                else:
                    continue
    except:
        email_message        =  ""
                

In [13]:
import re as regex

s_list      = ['Profile ID: M5929711 in Matrimony.com', 'ddd']

for i in range(len(s_list)):
    Profile_ID = ' '
    s          = s_list[i]
    try: 
        Profile_ID = re.search(r'Profile ID:(.*) in Matrimony.com', s).group(1).strip()
    except:
        Profile_ID = ''
        
    print(Profile_ID)

M5929711



In [14]:
from   datetime  import  datetime

now              =    datetime.now()
current_time     =    now.strftime("%H:%M:%S")
error            =    None

In [15]:
mem              =    Member("localhost", "testUser", "A4min@123", "matrimony" )
connection       =    mem.db_connection()
cursor           =    connection.cursor()

In [16]:
nrows            =    mails_df.shape[0]

In [17]:
nrows

58

In [18]:
mails_df.tail(3).T

,0,0,0
From,"""L T Realty"" <newsletters@greatmail.in>",LinkedIn Welcome Team <messages-noreply@linked...,Bajaj Finserv EMI Card <info@oristay.in>
To,pvs1301@gmail.com,Subramanian PV <pvs1301@gmail.com>,pvs1301@gmail.com
Date,"Wed, 25 Aug 2021 10:56:05","Wed, 25 Aug 2021 10:58:09","Wed, 25 Aug 2021 10:51:12"
Subject,"pvs1301@gmail.com, Spacious 1, 2 and 3 BHK Fla...",Take your conversations with you. Be notified ...,Get SUN-Sational EMI deals on latest Home Appl...
Profile ID,,,
Category,General,General,General


In [19]:
profile_id_list       =   mails_df['Profile ID'].tolist()

In [20]:
receiver_address_list =   mails_df['From'].tolist()

In [21]:
 #Create SMTP session for sending the mail
session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
session.starttls() #enable security
session.login(sender_address, sender_pass) #login with mail_id and password

(235, b'2.7.0 Accepted')

In [22]:
for  ind in range(len(profile_id_list)):
    
     profile_id       =  profile_id_list[ind]
        
     if  len(profile_id) > 3:
         print("IND {}".format(ind))
         print("\nProfile ID {}".format(profile_id))
        
         receiver_address =   receiver_address_list[ind]
            
         sql_select       =   """SELECT AGE, GENDER, MARITAL_STATUS, CASTE, SUB_CASTE, 
                                       STATE, CITY, COUNTRY, HEIGHT, PHYSICAL_STATUS, FAMILY_STATUS, 
                                       FAMILY_TYPE, FAMILY_VALUES, PARTNER_AGE, PARTNER_EDUCATION, 
                                       PARTNER_INCOME, PARTNER_LOCATION, ABOUT_DESCRIPTION 
                                 FROM MEMBER 
                                 WHERE PROFILE_ID = %s""" 
         cursor.execute(sql_select, (profile_id,))
         record = cursor.fetchall()
         for row in record:
            print("AGE = ", row[0], )
            print("GENDER = ", row[1])
            print("MARITAL_STATUS = ", row[2])
            print("CASTE  = ", row[3], "\n")
            mail_content  = 'Pls check your details : \n' + '<== Age = ' + str(row[0]) + ' Gender = ' + row[1] + "  MARITAL_STATUS = " + row[2] +\
            " CASTE  = " + row[3] + "====>\n"
            #
            message              =   MIMEMultipart()
            message['From']      =   sender_address
            message['To']        =   receiver_address
            message['Subject']   =   'An reply test mail sent by Python'   #The subject line
            #The body and the attachments for the mail
            message.attach(MIMEText(mail_content, 'plain'))
            text = message.as_string()
            session.sendmail(sender_address, receiver_address, text)

IND 39

Profile ID M5929711
AGE =  27
GENDER =  Female
MARITAL_STATUS =  Never married
CASTE  =  Mudaliar 



In [23]:
session.quit()
print('Mail Sent???')

Mail Sent???
